In [ ]:
### Fitness Koçu Yapay Zeka Uygulaması (LLM destekli)

# GEREKSİNİMLER
# pip install fastapi openai uvicorn

from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
import openai
import os

# OpenAI API anahtarını buraya gir (veya .env dosyasından çekebilirsin)
openai.api_key = "YOUR_API_KEY"

# FastAPI uygulaması oluştur
app = FastAPI()

# CORS ayarları (Her yerden istek almasına izin verir)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Kullanıcıdan gelen sohbet isteklerini ele al
@app.post("/chat")
async def chat(request: Request):
    data = await request.json()
    user_message = data.get("message", "")
    user_info = data.get("user", {})  # yaş, boy, kilo, hedef gibi bilgiler

    system_prompt = f"""
    Sen kişisel bir fitness koçusun.
    Kullanıcının bilgileri:
    - Yaş: {user_info.get('age', '?')}
    - Boy: {user_info.get('height', '?')} cm
    - Kilo: {user_info.get('weight', '?')} kg
    - Hedef: {user_info.get('goal', 'belirtilmedi')}

    Günlük egzersiz planı öner, basit ve evde yapılabilir olsun. Kullanıcıyı motive et.
    """

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )

    reply = response.choices[0].message.content
    return {"response": reply}

# Uygulamayı başlatmak için:
# uvicorn fitness_ai_coach:app --reload